In [79]:
import importlib
import ChessEnv
import ChessEngine
import network
import torch
import os

importlib.reload(ChessEnv)
importlib.reload(ChessEngine)
importlib.reload(network)

<module 'network' from 'd:\\User\\ProjectGithub\\hiepnguyenn-99\\Chess\\RL\\network.py'>

In [80]:
# khởi tạo môi trường và mô hình
env = ChessEnv.Env()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

print(f'device {device}')

print(f'env.action_size {env.action_size}')
q_net = network.DQN(env.action_size).to(device)
model_path = 'DQN.pth'
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    q_net.load_state_dict(checkpoint)
    print("Đã load model")
else:
    print("Không tìm thấy model")

device cpu
env.action_size 4032
Đã load model


In [81]:
def state_to_tensor(state, whiteToMove): # từ bàn cờ chuyển thành đầu vào cho DQN
    mapping = ["wp","wN","wB","wR","wQ","wK","bp","bN","bB","bR","bQ","bK"] 
    planes = torch.zeros((12, 8, 8), dtype=torch.float32)
    
    for r in range(8):
        for c in range(8):
            sq = state[r][c]
            if sq in mapping:
                planes[(mapping.index(sq), r, c)] = 1.0

    # side to move
    stm = torch.full((1, 8, 8), float(whiteToMove), dtype=torch.float32)
    return torch.cat([planes, stm], dim=0) # (13,8,8)

def legal_moves_mask(env, validmoves):
    legal_mask = torch.zeros(env.action_size, dtype=torch.bool)
    for m in validmoves:
        mid = m.moveID
        if mid in env.moveid_to_index:
            legal_mask[env.moveid_to_index[mid]] = True
    return legal_mask

In [86]:
def BestRLMove(gs):
    state = gs.board
    whiteToMove = gs.whiteToMove
    validmoves = gs.getValidMoves()

    # input
    state_tensor = state_to_tensor(state, whiteToMove)   
    # print(f'state_tensor {state_tensor}')
    print(state_tensor.shape)
    print(type(state_tensor))
    legal_mask = legal_moves_mask(env, validmoves)
    print(f'legal_mask {legal_mask}')
    print(legal_mask.shape)
    print(type(legal_mask))

    with torch.no_grad():
        q_values = q_net(state_tensor.unsqueeze(0).to(device)) # action_size
        print (f'q_values {q_values}')
        q_values[~legal_mask.unsqueeze(0)] = -torch.inf
        best_action_index = q_values.argmax()
        print(f'best_action {best_action_index.item()}')
        print(f'q_value max {q_values.squeeze(0)[best_action_index.item()]}')
    
    best_action_mid = env.index_to_moveid[best_action_index.item()]
    s = str(best_action_mid).zfill(4)
    sr, sc, er, ec = map(int, s)
    print(sr, sc, er, ec)
    return ChessEngine.Move((sr, sc), (er, ec), gs.board)

move = BestRLMove(env.gs)
print(move.moveID)

torch.Size([13, 8, 8])
<class 'torch.Tensor'>
legal_mask tensor([False, False, False,  ..., False, False, False])
torch.Size([4032])
<class 'torch.Tensor'>
q_values tensor([[nan, nan, nan,  ..., nan, nan, nan]])
best_action 3056
q_value max nan
6 0 4 0
6040
